In [1]:
# Step 0: Install Keras NLP
!pip install -q keras-nlp


In [2]:
import tensorflow as tf
import keras_nlp
from tensorflow.keras import layers
import numpy as np

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.18.0


In [3]:
# Step 2: Minimal synthetic dataset for binary classification
vocab = ["good", "bad", "awesome", "boring", "great", "terrible"]
word_index = {word: idx + 2 for idx, word in enumerate(vocab)}  # reserve 0 for pad, 1 for unk

def encode(text):
    return [word_index.get(word, 1) for word in text.lower().split()]

texts = [
    "good movie", "bad movie", "awesome experience", "boring film",
    "great story", "terrible direction"
]
labels = [1, 0, 1, 0, 1, 0]

X = tf.keras.preprocessing.sequence.pad_sequences([encode(t) for t in texts], maxlen=5)
y = np.array(labels)


In [4]:
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=len(word_index) + 2,
    sequence_length=5,
    embedding_dim=16
)

transformer_layer = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=32,
    num_heads=2
)

inputs = tf.keras.Input(shape=(5,), dtype="int32")
x = embedding_layer(inputs)
x = transformer_layer(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(16, activation="relu")(x)
outputs = layers.Dense(2)(x)

model = tf.keras.Model(inputs, outputs)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, 5, 16)          │           208 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, 5, 16)          │         2,224 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,738 (10.70 KB)

 Trainable params: 2,738 (10.70 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(X, y, epochs=3, batch_size=2, verbose=1)


Epoch 1/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.7292 - loss: 0.6672
Epoch 2/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9167 - loss: 0.6589
Epoch 3/3
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 0.6598


In [6]:
# Quick test inference
test_texts = ["awesome movie", "boring film", "great acting", "bad story"]
test_X = tf.keras.preprocessing.sequence.pad_sequences([encode(t) for t in test_texts], maxlen=5)

preds = model.predict(test_X)

for i, text in enumerate(test_texts):
    label = np.argmax(preds[i])
    sentiment = "Positive" if label == 1 else "Negative"
    confidence = tf.nn.softmax(preds[i])[label].numpy()
    print(f"Text: {text}")
    print(f"Prediction: {sentiment} (Confidence: {confidence:.2f})")
    print("-" * 40)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
Text: awesome movie
Prediction: Positive (Confidence: 0.56)
----------------------------------------
Text: boring film
Prediction: Negative (Confidence: 0.53)
----------------------------------------
Text: great acting
Prediction: Positive (Confidence: 0.53)
----------------------------------------
Text: bad story
Prediction: Negative (Confidence: 0.53)
----------------------------------------
